In [10]:
import pandas as pd
from pathlib import Path
from bitcoin_risk.risk import prepare_and_compute_risk

# complete data-range
start = pd.to_datetime('2015-08-03')
end = pd.to_datetime('2025-07-21')
future = pd.to_datetime('2034-11-30')
last_dates = ['2019-11-30', '2020-11-30', '2021-11-30', '2022-11-30', '2023-11-30', '2024-11-30']
# last_dates = ['2025-11-30']

for last_date in last_dates:
    # result directory
    current_dir = Path.cwd()
    # path = current_dir / "eth" / last_date
    path = current_dir / "docs" / "eth" / last_date
    if not path.exists():
        path.mkdir(parents=True) 

    df = prepare_and_compute_risk('data/KRAKEN_ETHUSD.csv', start, end, future, last_date)
    # display(df)
    from bitcoin_risk.plotter import create_figures
    fig1, fig2, fig3, fig4 = create_figures(df, path)
fig1.show(); fig2.show(); fig3.show(); fig4.show()

In [ ]:
import pandas as pd
from pathlib import Path
from bitcoin_risk.risk import prepare_and_compute_risk
from bitcoin_risk.risk import _unix_to_btc_time, _bitcoin_time
import numpy as np

# complete data-range
start = pd.to_datetime('2015-08-03')
end = pd.to_datetime('2024-11-30')
future = pd.to_datetime('2034-11-30')
last_date = '2024-11-30'

# result directory
current_dir = Path.cwd()
path = current_dir / "docs" / "eth" / last_date
if not path.exists():
    path.mkdir(parents=True) 

# btc = prepare_and_compute_risk('data/BTCUSD_2024_12_24.csv', start, end, future, last_date)
df = prepare_and_compute_risk('data/INDEX_ETHUSD.csv', start, end, future, last_date)

def _evaluate_fit(fit, date):
    # date -> btc-time
    x = pd.to_datetime(date)
    x = x.apply(lambda x: x.timestamp())
    x = _unix_to_btc_time(x)
    x = np.log(x)
    return fit[0] * np.exp(fit[0] * x + fit[1])


def btc_fit(btc, dates, price):
    """
    Performs a logscale linear fit on the Bitcoin price data to predict future prices.

    Parameters:
    - dates (pd.Series): A series of dates for which the Bitcoin prices are known.
    - price (pd.Series): A series of Bitcoin prices corresponding to the dates.

    Returns:
    - pd.Series: A series of predicted Bitcoin prices based on the logscale linear fit.
    """
    t = _bitcoin_time(dates)
    t = np.log(t)
    y = np.log(price)
    btc_fit = np.polyfit(t, y, 1)
    return _evaluate_fit(btc_fit, btc['date'])


df['ddt'] = ((df['fit'] / df['fit'].shift(365)) - 1) * 100.
display(df)

import matplotlib.pyplot as plt
plt.plot(df['date'], df['ddt'])
# plt.yscale('log')
plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2034-01-01'))
plt.ylim((1e1, 1e2))
plt.xlabel("Date")
plt.ylabel("Yearly %-Returns")
plt.show()